<a href="https://colab.research.google.com/github/sevinc-elif/ALSforRecommendationSystem/blob/main/ALSforRecommendationSystem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Spark ı Colab a ekleme

In [2]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.5.1/spark-3.5.1-bin-hadoop3.tgz
!tar xf spark-3.5.1-bin-hadoop3.tgz
!pip install -q findspark

In [3]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.1-bin-hadoop3"

In [4]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # Property used to format output tables better
spark


In [5]:
#veri setini aktarma
data=spark.read.option("delimiter",";").csv('/content/drive/MyDrive/BTK/MarketSales_20220215.csv',header=True,inferSchema=True)

In [6]:
#verinin şemasını inceleme
data.printSchema()

root
 |-- ID: string (nullable = true)
 |-- ITEMNAME: string (nullable = true)
 |-- DATE_: string (nullable = true)
 |-- AMOUNT: double (nullable = true)
 |-- PRICE: double (nullable = true)
 |-- LINENETTOTAL: double (nullable = true)
 |-- BRANCHNR: integer (nullable = true)
 |-- BRANCH: string (nullable = true)
 |-- SALESMAN: string (nullable = true)
 |-- CITY: string (nullable = true)
 |-- REGION: string (nullable = true)
 |-- CLIENTCODE: string (nullable = true)
 |-- CLIENTNAME: string (nullable = true)
 |-- BRANDCODE: string (nullable = true)
 |-- BRAND: string (nullable = true)
 |-- CATEGORY_NAME1: string (nullable = true)
 |-- CATEGORY_NAME2: string (nullable = true)
 |-- CATEGORY_NAME3: string (nullable = true)
 |-- STARTDATE: string (nullable = true)
 |-- ENDDATE: string (nullable = true)
 |-- GENDER: string (nullable = true)



In [7]:
#veriyi görüntüleme
data.show(truncate=False)

+-----+--------------------------------------+--------------+------+-----+------------+--------+----------------+------------------+---------+-----------------+----------+----------------+---------+--------------+---------------+---------------+------------------+--------------+--------------+------+
|ID   |ITEMNAME                              |DATE_         |AMOUNT|PRICE|LINENETTOTAL|BRANCHNR|BRANCH          |SALESMAN          |CITY     |REGION           |CLIENTCODE|CLIENTNAME      |BRANDCODE|BRAND         |CATEGORY_NAME1 |CATEGORY_NAME2 |CATEGORY_NAME3    |STARTDATE     |ENDDATE       |GENDER|
+-----+--------------------------------------+--------------+------+-----+------------+--------+----------------+------------------+---------+-----------------+----------+----------------+---------+--------------+---------------+---------------+------------------+--------------+--------------+------+
|11738|SPRITE 1 LT LIMON AROMALI GAZOZ       |07/01/17 00:00|1.0   |2.0  |2.0         |52     

In [8]:
#veri setindeki satır sayısı
data.count()

611116

In [9]:
# ITEMNAME ve CLIENTNAME sütunlarında boş olan satırları temizleme
data = data.dropna(subset=['ITEMNAME', 'CLIENTNAME'])


In [10]:
#temizlenen veri setindeki satır sayısı
data.count()

585796

In [11]:
#string değerler içerdiği için müşteri kodu sütununu silme
data=data.drop('CLIENTCODE')

In [12]:
#pandas işlemleri için kütüphaneyi import etme
import pandas as pd

In [13]:
#veriyi pandas a çevirme
data = data.toPandas()

# ITEMNAME sütununa dayalı benzersiz bir ITEMCODE sütunu oluşturma
data['ITEMCODE'] = pd.factorize(data['ITEMNAME'])[0]

# Yeni veri setini görüntüleme
print(data[['ITEMNAME', 'ITEMCODE']])


                               ITEMNAME  ITEMCODE
0       SPRITE 1 LT LIMON AROMALI GAZOZ         0
1                             TOZ SEKER         1
2                  FALIM SAKIZ 5LI NANE         2
3                  FALIM SAKIZ 5LI NANE         2
4                 FALIM SAKIZ 5LI CILEK         3
...                                 ...       ...
585791                     EKMEK 250 GR       436
585792                     EKMEK 250 GR       436
585793                     EKMEK 250 GR       436
585794                     EKMEK 250 GR       436
585795           ETI 80 GR KARAM BITTER       403

[585796 rows x 2 columns]


In [14]:
# CLIENTNAME sütununa dayalı benzersiz bir CLIENTCODE sütunu oluşturma
data['CLIENTCODE'] = pd.factorize(data['CLIENTNAME'])[0]

# Yeni veri setini görüntüleme
print(data[['CLIENTNAME', 'CLIENTCODE']])

              CLIENTNAME  CLIENTCODE
0         Sercan KIZILOK           0
1           �smet �INGIR           1
2            Ya��z KUBAL           2
3            Ya��z KUBAL           2
4           Habibe AYSAN           3
...                  ...         ...
585791   Mahmut BAYATG�L       10480
585792       Gamze BOTU�       36105
585793  Cengiz BEREKET��       32954
585794  Elifnur EG�NKAYA       48454
585795    Hacer CANDEM�R       35802

[585796 rows x 2 columns]


In [15]:
#veriyi görüntüleme
data.head()

,ID,ITEMNAME,DATE_,AMOUNT,PRICE,LINENETTOTAL,BRANCHNR,BRANCH,SALESMAN,CITY,...,BRANDCODE,BRAND,CATEGORY_NAME1,CATEGORY_NAME2,CATEGORY_NAME3,STARTDATE,ENDDATE,GENDER,ITEMCODE,CLIENTCODE
0,11738,SPRITE 1 LT LIMON AROMALI GAZOZ,07/01/17 00:00,1.0,2.00,2.00,52,Kocaeli Subesi,Ey�p C�NE,Kocaeli,...,156,SPR�TE,��ECEK,GAZLI ��ECEK,GAZOZ,08/01/17 16:16,08/01/17 16:17,E,0,0
1,10537,TOZ SEKER,06/01/17 00:00,5.0,2.65,13.25,8,Antalya Subesi,�lhan �RENL�,Antalya,...,None,None,��ECEK,�AY KAHVE,SEKER TATLANDIRICI,07/01/17 11:04,07/01/17 11:05,E,1,1
2,11335,FALIM SAKIZ 5LI NANE,03/01/17 00:00,1.0,0.40,0.40,40,�stanbul Subesi,�smet SARTIK,�stanbul,...,300,FALIM,GIDA,SAKIZ SEKERLEME,SAKIZ,04/01/17 14:00,04/01/17 14:01,E,2,2
3,11336,FALIM SAKIZ 5LI NANE,03/01/17 00:00,1.0,0.40,0.40,40,�stanbul Subesi,�smet SARTIK,�stanbul,...,300,FALIM,GIDA,SAKIZ SEKERLEME,SAKIZ,04/01/17 14:00,04/01/17 14:01,E,2,2
4,10115,FALIM SAKIZ 5LI CILEK,05/01/17 00:00,1.0,0.40,0.40,41,�zmir Subesi,G�lten I�LAK,�zmir,...,300,FALIM,GIDA,SAKIZ SEKERLEME,SAKIZ,06/01/17 14:00,06/01/17 14:01,K,3,3


In [16]:
#ALS algoritması ve doğruluk değeri için gerekli kütüphaneleri import etme
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator

In [17]:
#veriyi tekrar spark a çevirme
data = spark.createDataFrame(data)

In [18]:
#verideki benzersiz müşteri sayısı
data.select("CLIENTCODE").distinct().count()

52703

In [19]:
#verideki benzersiz item sayısı
data.select("ITEMCODE").distinct().count()

9266

In [44]:
# Müşteri ve ürün bazında satın alma sayısını hesaplama
rating_df = data.groupBy("CLIENTCODE", "ITEMCODE").count().withColumnRenamed("count", "RATING")

# Mevcut DataFrame ile birleştirerek yeni sütunu ekleme
data = data.join(rating_df, on=["CLIENTCODE", "ITEMCODE"], how="left")
data.show()

+----------+--------+------+--------------------+--------------+------+-----+------------+--------+-----------------+------------------+----------+-----------------+------------------+---------+---------+---------------+---------------+------------------+--------------+--------------+------+------+
|CLIENTCODE|ITEMCODE|    ID|            ITEMNAME|         DATE_|AMOUNT|PRICE|LINENETTOTAL|BRANCHNR|           BRANCH|          SALESMAN|      CITY|           REGION|        CLIENTNAME|BRANDCODE|    BRAND| CATEGORY_NAME1| CATEGORY_NAME2|    CATEGORY_NAME3|     STARTDATE|       ENDDATE|GENDER|RATING|
+----------+--------+------+--------------------+--------------+------+-----+------------+--------+-----------------+------------------+----------+-----------------+------------------+---------+---------+---------------+---------------+------------------+--------------+--------------+------+------+
|         1|       1| 10537|           TOZ SEKER|06/01/17 00:00|   5.0| 2.65|       13.25|       8| 

In [50]:
#bir müşterinin bir ürünü en fazla kaç kere aldığını gösterme
data.select("RATING").distinct().count()

14

In [45]:
#Veri setinden algoritmada kullanılacak ilgili alanları yeni bir veri setine atama
data_=data.select("CLIENTCODE","ITEMCODE","RATING")

In [46]:
#Bu veri setini eğitim ve test olarak bölme
trainDF,testDF=data_.randomSplit([0.8,0.2])

In [32]:
#eğitim veri sayısı
trainDF.count()

468593

In [33]:
#test veri sayısı
testDF.count()

117203

In [48]:
#ALS fonksiyonuna gerekli parametreleri verme
als=ALS(userCol="CLIENTCODE",itemCol="ITEMCODE",ratingCol="RATING",coldStartStrategy="drop",nonnegative=True)

In [49]:
#Modeli eğitme
model=als.fit(trainDF)

In [51]:
#Test veri setinden tahminler oluşturma
predict=model.transform(testDF)

In [52]:
#Tahminleri görüntüleme
predict.show()

+----------+--------+------+----------+
|CLIENTCODE|ITEMCODE|RATING|prediction|
+----------+--------+------+----------+
|       463|     411|     1| 1.0109797|
|       471|     422|     2| 1.3669574|
|       471|     644|     1| 1.6594344|
|       471|     772|     1| 1.1131002|
|       471|    1104|     3|  1.520147|
|       471|    1193|     1| 0.9181811|
|       496|     422|     2| 1.3322736|
|       496|     804|     1| 1.2149088|
|       833|      44|     1| 1.1818179|
|      1088|      55|     1|0.87515235|
|      1088|    1534|     1|0.86440444|
|      1088|    3895|     1| 1.1466732|
|      1238|      61|     1| 1.0293196|
|      1238|     345|     1| 1.0350469|
|      1238|    5028|     1|  1.361135|
|      1342|    2023|     1| 1.2323959|
|      1580|      46|     1| 0.9653956|
|      1580|     259|     1| 1.1444203|
|      1580|     614|     1| 1.0797535|
|      1580|    3133|     1| 0.9721114|
+----------+--------+------+----------+
only showing top 20 rows



In [53]:
#Modelin doğruluğunu inceleme
evaluator=RegressionEvaluator(metricName="rmse",labelCol="RATING",predictionCol="prediction")

In [54]:
rmse=evaluator.evaluate(predict)

In [55]:
#hata oranı
rmse

0.4075867330420228

In [58]:
#hata oranının büyüklüğünü anlayabilmek için ratinglerin ortalamasının yazdırılması
data.select('RATING').agg({'RATING':'mean'}).show()

+------------------+
|       avg(RATING)|
+------------------+
|1.2486155590000614|
+------------------+



In [57]:
#Her bir müşteriye özel 3 ürün önerisi
model.recommendForAllUsers(numItems=3).show(truncate=False)

+----------+---------------------------------------------------------+
|CLIENTCODE|recommendations                                          |
+----------+---------------------------------------------------------+
|1         |[{9150, 2.9310951}, {8061, 2.9172232}, {3765, 2.7226257}]|
|3         |[{5970, 2.4674907}, {9150, 2.4522004}, {8061, 2.4387717}]|
|5         |[{5433, 3.6706877}, {2576, 3.4692152}, {3765, 3.0720844}]|
|6         |[{8061, 3.2314706}, {4784, 3.0319357}, {9150, 2.843273}] |
|9         |[{8061, 3.6143997}, {1123, 3.3655148}, {5970, 3.2508407}]|
|12        |[{8061, 2.5666249}, {9150, 2.4917092}, {3765, 2.3000653}]|
|13        |[{8061, 2.4886265}, {9150, 2.405911}, {3765, 2.22077}]   |
|15        |[{9150, 2.37844}, {8061, 2.236093}, {3765, 2.124254}]    |
|16        |[{8061, 4.3504963}, {9150, 3.633235}, {9002, 3.567014}]  |
|17        |[{9150, 3.5915532}, {8061, 3.4470344}, {3765, 3.2099156}]|
|19        |[{3025, 4.9095936}, {2498, 4.3756824}, {9150, 4.3262024}]|
|20   